In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, mae, rmse
import os
import csv

def mbe(y_true, y_pred):
    return np.mean(y_true - y_pred)

folder_path = 'SeaData\\csv\\d10'
date = 'd10'

with open(f"Statistics for Time Series III - ETS-Itterative {date}.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([f'Statistics for Time Series III - ETS-Itterative {date}'])
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        print(f'Processing file: {file_path}')
        tag = ''
        if 'dbiastg' in file_name:
            tag = 'dbiastg'
        else:
            tag = 'biastg'   
        writer.writerow([file_name])       

        data = pd.read_csv(file_path, index_col='date', parse_dates=True)

        # Prepare the data
        data = data[tag]
        data = data.asfreq('B').fillna(method='ffill')  # Fill missing values

        # Create a TimeSeries object
        ts = TimeSeries.from_dataframe(data.reset_index(), 'date', tag)

        # Normalize the dataset
        scaler = Scaler()
        ts_scaled = scaler.fit_transform(ts)

        # Calculate the split date for 80% of the data
        split_date = data.index[int(0.8 * len(data))]

        # Split the data into training and testing sets
        train_ts, test_ts = ts_scaled.split_after(split_date)

        # Create and fit the ETS model
        model = ExponentialSmoothing()
        model.fit(train_ts)

        # Make predictions
        predicted_ts = model.predict(len(test_ts))

        # Invert scaling
        predicted_ts_unscaled = scaler.inverse_transform(predicted_ts)
        test_ts_unscaled = scaler.inverse_transform(test_ts)

        # Evaluate the model
        print("RMSE: {:.2f}".format(rmse(test_ts_unscaled, predicted_ts_unscaled)))
        writer.writerow(['RMSE',rmse(test_ts_unscaled, predicted_ts_unscaled)])
        print("MAE: {:.2f}".format(mae(test_ts_unscaled, predicted_ts_unscaled)))
        writer.writerow(['MAE',mae(test_ts_unscaled, predicted_ts_unscaled)])
        print("MBE: {:.2f}".format(mbe(test_ts_unscaled.values(), predicted_ts_unscaled.values())))
        writer.writerow(['MBE',mbe(test_ts_unscaled.values(), predicted_ts_unscaled.values())])
        print("MAPE: {:.2f}%".format(mape(test_ts_unscaled, predicted_ts_unscaled)))
        writer.writerow(['MAPE',mape(test_ts_unscaled, predicted_ts_unscaled)])



        # Plot the results
        plt.figure(figsize=(14, 6))
        ts.plot(label='Observed')
        predicted_ts_unscaled.plot(color='r', label='Predicted')
        plt.xlabel('Date')
        plt.ylabel(tag)
        plt.legend()
        plt.show()

In [ ]:
from darts.models import ExponentialSmoothing
from darts.backtesting import (
    backtest_forecasting,
    plot_residuals_analysis,
)

# Additional imports
import warnings
warnings.filterwarnings("ignore")

# Define ETS model types
ets_model_types = [('add', 'add', 'add', None),
                   ('mul', 'add', 'add', None),
                   ('add', 'add', 'mul', None),
                   ('mul', 'add', 'mul', None)]

# Initialize minimum error and best model
min_error = float('inf')
best_model = None
best_model_type = None

# Loop through ETS model types
for model_type in ets_model_types:
    print(f"Trying ETS model with error type: {model_type}")

    # Create the ETS model
    model = ExponentialSmoothing(*model_type)

    # Use rolling window to create multiple training and testing sets and backtest the model
    errors = backtest_forecasting(ts_scaled,
                                  model,
                                  pd.Timestamp(split_date),
                                  fcast_horizon_n=1,  # 1-step ahead forecasting
                                  verbose=True)

    # Calculate the mean error
    mean_error = np.mean(errors)

    # Check if the current model has a lower error than the previous models
    if mean_error < min_error:
        min_error = mean_error
        best_model = model
        best_model_type = model_type

print(f"Best ETS model: {best_model_type} with mean error: {min_error}")

# Fit the best model on the training set
best_model.fit(train_ts)

# Make predictions
predicted_ts = best_model.predict(len(test_ts))

# Invert scaling
predicted_ts_unscaled = scaler.inverse_transform(predicted_ts)
test_ts_unscaled = scaler.inverse_transform(test_ts)

# Evaluate the best model
print("RMSE: {:.2f}".format(rmse(test_ts_unscaled, predicted_ts_unscaled)))
print("MAE: {:.2f}".format(mae(test_ts_unscaled, predicted_ts_unscaled)))
print("MBE: {:.2f}".format(mbe(test_ts_unscaled.values(), predicted_ts_unscaled.values())))
print("MAPE: {:.2f}%".format(mape(test_ts_unscaled, predicted_ts_unscaled)))

# Plot the results
plt.figure(figsize=(14, 6))
ts.plot(label='Observed')
predicted_ts_unscaled.plot(color='r', label='Predicted')
plt.xlabel('Date')
plt.ylabel(tag)
plt.legend()
plt.show()
